# <span style="font-family:PT Sans Narrow; color:orange; font-size:1.31em"> Main Idea: </span>

In [3]:
%reload_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('../../')

import tensorflow as tf

# setup tensorflow 2.4.1 to only use GPU memory as needed
physical_devices = tf.config.experimental.list_physical_devices('GPU')
for physical_device in physical_devices:
    tf.config.experimental.set_memory_growth(physical_device, True)

import funcs 
import load_data

import mlflow
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')


%reload_ext load_data
%reload_ext funcs

/opt/homebrew/Caskroom/miniforge/base/envs/paper_miniforge/lib/python3.8/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/opt/homebrew/Caskroom/miniforge/base/envs/paper_miniforge/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-07-01 00:31:20.248176: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-01 00:31:20.248275: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### <span style="font-family:PT Sans Narrow; font-size:1em"> MLFlow Setup </span>

In [4]:
aim1_1 = funcs.AIM1_1( experiment_name='label_inter_dependence'  ,  run_name='test'  ,  new_run=False  ,  dataset='cifar100' )

Connecting to the server...






************************************************
Access is restricted to AUTHORIZED USERS only! If
you are not authorized LEAVE NOW!
************************************************





Killing all active runs...
Setting up the experiment...
setting the tracking URI
setting/creating the experiment
Running the session...


In [ ]:
p = aim1_1.mlflow_ui(VIEW_PORT=6789)

### <span style="font-family:PT Sans Narrow; font-size:1em"> Loading the data  </span>

In [6]:
data_loader = load_data.LOAD_CIFAR100(site='local')

data_loader.load( data_modes=['train', 'valid', 'test'] , label_type='merged' )

data_loader.get_iterators(data_modes=['train'] , be_augmented=True  , Keras=True, TF=False, batch_size=128)
data_loader.get_iterators(data_modes=['valid'] , be_augmented=False, Keras=True, TF=False, batch_size=128)

### <span style="font-family:PT Sans Narrow; font-size:1em"> Getting the optimized model from server  </span>

In [17]:
model = aim1_1.get_model(data_loader=data_loader, optimize_model=False , epochs=1)

### <span style="font-family:PT Sans Narrow; font-size:1em"> Getting the optimization values </span>

In [18]:
x, y = np.array( data_loader.dataframes['valid'].data.to_list())  ,  np.array( data_loader.dataframes['valid'].labels.to_list()) 

df_score = aim1_1.get_optimization_values(x=x ,y=y , labels=data_loader.label_names )

df_score.head()

2022-07-01 00:43:30.081285: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-01 00:43:31.155255: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,loss_avg,acc_avg,pred,pred_binary,truth,loss
0,0.037178,0.991667,"[0.002931992, 0.011508828, 0.0045414907, 0.000...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...","[0.0029361595, 0.011575413, 0.0045517334, 0.00..."
1,0.028374,0.983333,"[0.0037598696, 0.022452775, 0.00077427167, 0.0...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...","[0.0037668347, 0.02270851, 0.00077440444, 0.00..."
2,0.072949,0.983333,"[0.004610302, 0.016615499, 0.023130799, 0.0030...","[False, False, False, False, False, False, Fal...","[False, False, True, False, False, False, Fals...","[0.0046208114, 0.016754966, 3.7665858, 0.00302..."
3,0.065633,0.983333,"[0.014075292, 0.009186894, 0.0044198986, 0.002...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...","[0.014175105, 0.009229229, 0.0044296067, 0.002..."
4,0.041336,0.983333,"[0.0011161854, 0.005218171, 0.08704416, 0.0211...","[False, False, False, False, False, False, Fal...","[False, False, False, False, True, False, Fals...","[0.0011166562, 0.005231692, 0.091067635, 0.021..."


In [19]:
data_loader.dataframes['valid'].labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### <span style="font-family:PT Sans Narrow; font-size:1em">  Measuring updated results for all proposed techniques </span>

In [20]:
updated_values, mean_values, accuracies = aim1_1.measure_new_predicted_probabilities(hierarchy=data_loader.hierarchy, weight=1, bias=0.5, METRIC_USED_TO_MEASURE_COEFFICIENT='pred', MODE_UPDATE_LOSS='pred', THRESHOLD_UPDATE_LOSS=0.5)

accuracies.round(decimals=2)

,original,new
aquatic_mammals,0.95,0.95
fish,0.95,0.95
flowers,0.96,0.96
food_containers,0.95,0.95
fruit_and_vegetables,0.96,0.96
...,...,...
whale,0.99,0.99
willow_tree,0.99,0.99
wolf,0.99,0.99
woman,0.99,0.99


In [21]:
mean_values

,loss,truth,pred,pred_new
aquatic_mammals,0.18,0.05,0.02,0.02
fish,0.15,0.05,0.03,0.03
flowers,0.11,0.05,0.03,0.03
food_containers,0.15,0.05,0.02,0.02
fruit_and_vegetables,0.13,0.05,0.03,0.03
...,...,...,...,...
whale,0.05,0.01,0.00,0.00
willow_tree,0.04,0.01,0.01,0.00
wolf,0.04,0.01,0.00,0.00
woman,0.04,0.01,0.01,0.00


### <span style="font-family:PT Sans Narrow; font-size:1em">  Checking the results per subject </span>

In [22]:
subjcet_index = 3

data_subject = {}
for name in ['coefficient' ,'loss' , 'truth' , 'pred' , 'pred_new']:
    data_subject[name] = updated_values[name].iloc[subjcet_index]

pd.DataFrame(data_subject).round(decimals=4)

,coefficient,loss,truth,pred,pred_new
aquatic_mammals,1.0000,0.0142,0,0.0141,0.0141
fish,1.0000,0.0092,0,0.0092,0.0092
flowers,1.0000,0.0044,0,0.0044,0.0044
food_containers,1.0000,0.0024,0,0.0024,0.0024
fruit_and_vegetables,1.0000,0.0058,0,0.0057,0.0057
...,...,...,...,...,...
whale,0.5141,0.0005,0,0.0005,0.0002
willow_tree,0.5014,0.0007,0,0.0007,0.0004
wolf,0.5973,0.0100,0,0.0099,0.0059
woman,0.5050,0.0107,0,0.0107,0.0054


# <span style="color:orange; font-family:PT Sans Narrow; font-size:1em"> killing the mlflow & ssh sessions </span>

In [18]:
# # closing the child mlflow session
# aim1_1.cleanup_mlflow_after_runs()

# # closing the ssh session
# aim1_1.ssh_session.kill()